In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon July 16 10:08:53 2018

@author: sxx
"""
import time
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import traceback 

from predict import load_data as load_data
from predict import transform_data as transform_data
from predict import format_data as format_data
from predict import divide_data as divide_data
from predict import LstmModel as LstmModel

Using TensorFlow backend.
/opt/anaconda3/envs/health/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
current_time=time.strftime('%Y-%m-%d-%H-%M',time.localtime())

def experiments(params):
    print ("epoch:",params['epoch']," batch_size: ",params['batch'])
    csvfile = './data/health_1799.csv'
    x,y = load_data(csvfile)
    x_,y_ = transform_data(x,y,False)
    datax,datay = format_data(x_,y_)
    trainX,trainY,validX,validY,testX,testY = divide_data(datax,datay)
    
    try:
        ##调试模型epoch,batch_size,learnrate
        #model = LstmModel(epoches=params['epoch'], batch_size=params['batch'],lr=params['lr'])

        ##测试模型units,dropout_rate
        model = LstmModel(units=params['units'],dr=params['dr'],lr=0.01,epoches=10,batch_size=4)
        health_model = model.create_model()
        save_path = './models/health_{0}.h5'.format(current_time)
        model.train_model_1(trainX,trainY,validX,validY,save_path)
        testPredict = health_model.predict(testX)
        print (testY,"-->",testPredict)
    except Exception as e:
        print ("something happen:",repr(e))
        print ('-' * 20)
        with open("./data/err.log","a") as logf:
            traceback.print_exc(file=logf)
        return {'loss': 999999, 'status': STATUS_OK}
    
    mse = np.mean(np.square(testPredict - testY))
    print ("mse:",mse)
    print ('-' * 20)

    return {'loss': mse, 'status': STATUS_OK}


In [3]:
if __name__ == '__main__':
    space = {
        'units':{
        'ls_units1': hp.quniform('ls_units1',12,16,4),
        'ls_units2': hp.quniform('ls_units2',8,12,4),
        'ls_units3': hp.quniform('ls_units3',8,12,4),
        'ls_units4': hp.quniform('ls_units4',4,8,4),      
        'ds_units5': hp.quniform('ds_units5',6,8,2)},
        'dr': hp.uniform('dr',0.1,0.6),
        'batch': hp.randint('batch',256),
        'epoch': hp.randint('epoch',500),
        'lr': hp.uniform('lr',0.0001,0.01),
        'activation': hp.choice('activation',['relu',
                                                'sigmoid',
                                                'tanh',
                                                'linear'])
        }
    
    trials = Trials()
    best = fmin(experiments, space, algo=tpe.suggest, max_evals=5, trials=trials)
    print ('best: ',best)
    print ("hyperoptparam end")

epoch: 7  batch_size:  156
LSTM model unit1_size:12.0 unit2_size:8.0 unit3_size:12.0 unit4_size:4.0 unit5_size:8.0 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 15, 12)            3120      
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 12)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 12)            48        
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 12)            1200      
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 12)            0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 12)            48        
_________________________

69/69 [==============================] - 1s 10ms/step - loss: 0.0040 - val_loss: 0.0041

Epoch 00008: val_loss improved from 0.00408 to 0.00408, saving model to ./models/health_checkpoint_2018-08-03-10-11.hdf5
Epoch 9/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0041 - val_loss: 0.0040

Epoch 00009: val_loss improved from 0.00408 to 0.00402, saving model to ./models/health_checkpoint_2018-08-03-10-11.hdf5
Epoch 10/10
69/69 [==============================] - 1s 10ms/step - loss: 0.0035 - val_loss: 0.0040

Epoch 00010: val_loss improved from 0.00402 to 0.00400, saving model to ./models/health_checkpoint_2018-08-03-10-11.hdf5
[[0.975 0.96  0.97  0.975 0.96 ]
 [0.96  0.97  0.975 0.96  0.96 ]
 [0.97  0.975 0.96  0.96  0.95 ]
 [0.975 0.96  0.96  0.95  0.68 ]
 [0.96  0.96  0.95  0.68  0.97 ]
 [0.96  0.95  0.68  0.97  0.95 ]
 [0.95  0.68  0.97  0.95  0.96 ]
 [0.68  0.97  0.95  0.96  0.965]
 [0.97  0.95  0.96  0.965 0.97 ]
 [0.95  0.96  0.965 0.97  0.97 ]] --> [[0.9542364 

69/69 [==============================] - 1s 10ms/step - loss: 0.0097 - val_loss: 0.0082

Epoch 00005: val_loss did not improve from 0.00363
Epoch 00005: early stopping
[[0.975 0.96  0.97  0.975 0.96 ]
 [0.96  0.97  0.975 0.96  0.96 ]
 [0.97  0.975 0.96  0.96  0.95 ]
 [0.975 0.96  0.96  0.95  0.68 ]
 [0.96  0.96  0.95  0.68  0.97 ]
 [0.96  0.95  0.68  0.97  0.95 ]
 [0.95  0.68  0.97  0.95  0.96 ]
 [0.68  0.97  0.95  0.96  0.965]
 [0.97  0.95  0.96  0.965 0.97 ]
 [0.95  0.96  0.965 0.97  0.97 ]] --> [[0.8989924  0.8862237  0.9004698  0.8802644  0.90294343]
 [0.8923327  0.8792666  0.89879435 0.86829555 0.8963691 ]
 [0.8919012  0.8780094  0.8963788  0.86982894 0.8965804 ]
 [0.89803916 0.8842511  0.90036446 0.878168   0.9021494 ]
 [0.897093   0.8831366  0.90318793 0.87623185 0.90051323]
 [0.89517945 0.8817818  0.8994697  0.87650025 0.8984996 ]
 [0.8897173  0.87769955 0.8897601  0.86843824 0.89423406]
 [0.8893528  0.8781531  0.8847777  0.8670679  0.89482677]
 [0.8874328  0.87640744 0.8842455

In [5]:
import hyperopt
print (hyperopt.space_eval(space, best))

{'activation': 'tanh', 'batch': 156, 'dr': 0.26647266397637004, 'epoch': 7, 'lr': 0.0028688204933912578, 'units': {'ds_units5': 8.0, 'ls_units1': 12.0, 'ls_units2': 8.0, 'ls_units3': 12.0, 'ls_units4': 4.0}}
